In [7]:
import os
import csv
from tokenizers import Tokenizer
import openai
import requests

In [8]:
def count_tokens(text):
    global tokenizer
    if 'tokenizer' not in globals():
        tokenizer = Tokenizer.from_pretrained("gpt2")
    return len(tokenizer.encode(text).tokens)

In [9]:
openai.api_key = ''

In [10]:
def get_embedding(text):
    retries = 0
    while retries <= 3:
        try:
            response = openai.Embedding.create(
                model="text-embedding-ada-002",
                input=text
            )
            embedding = response['data'][0]['embedding']
            return embedding
        except requests.exceptions.RequestException as e:
            retries += 1
            print(f"An error occurred: {e}. Retrying...")
            time.sleep(10)
    print("Failed to get embedding after 3 retries")
    return None

In [28]:
import time

with open('embeddings4.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['BondDetails','BondDetails_Tokens','BondDetails_Embedding'])
    with open('Fidelity.csv', newline='') as input_csvfile:
        input_csv_reader = csv.DictReader(input_csvfile)
          
        for row in input_csv_reader:
            bonddetails = row['Cusip'] + row['State']+row['Description']+row['Coupon']+row['Maturity Date']+row['Moody Rating']+row['S&P Rating']+row['Price Bid']+row['Yield Bid']+row['Ask Yield to Worst']+row['Ask Yield to Maturity']+row['Quantity Bid(min)']+row['Quantity Ask(min)']+row['Attributes']
            bonddetails_tokens = count_tokens(bonddetails)
            bonddetails_embedding = get_embedding(bonddetails)

            csv_writer.writerow([bonddetails,  bonddetails_tokens, bonddetails_embedding])
            print(f"DONE with row {input_csv_reader.line_num - 1}")

            time.sleep(1)

DONE with row 1
DONE with row 2
DONE with row 3
DONE with row 4
DONE with row 5
DONE with row 6
DONE with row 7
DONE with row 8
DONE with row 9
DONE with row 10
DONE with row 11
DONE with row 12
DONE with row 13
DONE with row 14
DONE with row 15
DONE with row 16
DONE with row 17
DONE with row 18
DONE with row 19
DONE with row 20
DONE with row 21
DONE with row 22
DONE with row 23
DONE with row 24
DONE with row 25
DONE with row 26
DONE with row 27
DONE with row 28
DONE with row 29
DONE with row 30
DONE with row 31
DONE with row 32
DONE with row 33
DONE with row 34
DONE with row 35
DONE with row 36
DONE with row 37
DONE with row 38
DONE with row 39
DONE with row 40
DONE with row 41
DONE with row 42
DONE with row 43
DONE with row 44
DONE with row 45
DONE with row 46
DONE with row 47
DONE with row 48
DONE with row 49
DONE with row 50
DONE with row 51
DONE with row 52
DONE with row 53
DONE with row 54
DONE with row 55
DONE with row 56
DONE with row 57
DONE with row 58
DONE with row 59
DONE w

TypeError: can only concatenate str (not "NoneType") to str

In [2]:
import numpy as np
from numpy.linalg import norm
import ast

def cosine_similarity(X, Y):
    A = np.array(ast.literal_eval(X))
    B = np.array(Y)
    cosine = np.dot(A,B)/(norm(A)*norm(B))

    return cosine

In [3]:
def get_vector_similarity(q_embedding):

    similarity_array = []

    with open('embeddings4.csv', newline='') as input_csvfile:
        input_csv_reader = csv.DictReader(input_csvfile)

        for row in input_csv_reader:
            score = cosine_similarity(row['BondDetails_Embedding'], q_embedding)
            
            similarity_array.append({
                "text": row["BondDetails"],
                "similarity_score": score,
                "tokens": int(row['BondDetails_Tokens'], 0)
            })

    similarity_array_filtered = [item for item in similarity_array if item.get("similarity_score") is not None]
    similarity_array_filtered = sorted(similarity_array_filtered, key=lambda item: item.get("similarity_score", 0), reverse=True)
            
    return similarity_array_filtered



In [11]:
MAX_SECTION_LEN = 500
tokenizer = Tokenizer.from_pretrained("gpt2")

def get_chosen_section(vector_similarity):
    chosen_sections = []
    chosen_sections_len = 0
    separator_len = 3

    for item in vector_similarity:
        text = item["text"]
        tokens = item["tokens"]
        encoded_text_ids = tokenizer.encode(text).ids

        tmp_count = tokens + separator_len

        if chosen_sections_len + tmp_count > MAX_SECTION_LEN:
            space_left = MAX_SECTION_LEN - chosen_sections_len - separator_len
            sliced_text_ids = encoded_text_ids[:space_left]
            chosen_sections.append(tokenizer.decode(sliced_text_ids))
            break

        chosen_sections_len += tmp_count
        chosen_sections.append(text)

    return chosen_sections


In [12]:
HEADER = "\"\"These are questions and answers. Please keep your answers to three sentences maximum, and speak in complete sentences. Stop speaking once your point is made.\n\nContext that may be useful:\n\"\""

QUESTIONS = [{'question': 'Give me top 3 bonds based on price per $100.',
'answer': 'The top 3 bonds based on price per $100 are:\n\n1. 912803BC6 - UNITED STATES TREAS BD STRIPP ZERO CPN 0.00000% 08/15/2023\n2. 91282CFQ9 - UNITED STATES TREAS SER BJ-2024 4.37500% 10/31/2024 NTS NOTE\n3. 912810ES3 - UNITED STATES TREAS BDS 7.50000% 11/15/2024'},
{'question': 'What is the date of maturity and price per $100 for the bond with CUSIPid = 912796ZF9?',
'answer': 'The bond with CUSIPid = 912796ZF9 has a date of maturity on 05/25/2023 and a price per $100.'},
{'question': 'What are the two bonds that have the highest price per $95?',
'answer': 'The two bonds with the highest price per $95 are:\n\n1. 912797FG7 - UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/30/2023\n2. 912796ZF9 - UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/25/2023'},
{'question': 'Which bond has the closest date of maturing from May 2023?',
'answer': 'The bonds that have the closest date of maturing from May 2023 are:\n\n1. UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/18/2023\n2. UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/23/2023'},
{'question': 'Which bond has the price ask as 99.974?',
'answer': 'The bond with a price ask of 99.974 is UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/18/2023.'}]


In [13]:
def construct_prompt(question, sections):
    q_and_a_string = ""

    for q in QUESTIONS:
        q_and_a_string += f"\n\n\nQ: {q['question']}\n\nA: {q['answer']}"

    return HEADER + sections + q_and_a_string + f"\n\n\nQ: {question}\n\nA: "


In [14]:
SEPARATOR = "\n* "

def ask(question):
    q_embedding = get_embedding(question)
    
    vector_similarity = get_vector_similarity(q_embedding)
    chosen_sections = SEPARATOR.join(get_chosen_section(vector_similarity))

    prompt = construct_prompt(question, chosen_sections)

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      max_tokens=150,
      temperature=0
    )

    return chosen_sections


In [70]:
ask(" Give me top 3 bonds based on price per $100")


<OpenAIObject text_completion id=cmpl-7GorolHhwF3IY0cjDfjg5aTFTSh0T at 0x16a003f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The top 3 bonds based on price per $100 are:\n\n1. 912803BC6 - UNITED STATES TREAS BD STRIPP ZERO CPN 0.00000% 08/15/2023\n2. 91282CFQ9 - UNITED STATES TREAS SER BJ-2024 4.37500% 10/31/2024 NTS NOTE\n3. 912810ES3 - UNITED STATES TREAS BDS 7.50000% 11/15/2024"
    }
  ],
  "created": 1684243136,
  "id": "cmpl-7GorolHhwF3IY0cjDfjg5aTFTSh0T",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 929,
    "total_tokens": 1029
  }
}

In [71]:
ask("Tell me about 91282CCD1")

<OpenAIObject text_completion id=cmpl-7GovXlyZ4EvSSDVWAzrafhjxoCcCS at 0x136cdc540> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " 91282CCD1 is a bond issued by the United States Treasury with a coupon rate of 4.37500%, maturing on 10/31/2024, and a price per $100 of 97.974."
    }
  ],
  "created": 1684243367,
  "id": "cmpl-7GovXlyZ4EvSSDVWAzrafhjxoCcCS",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 47,
    "prompt_tokens": 949,
    "total_tokens": 996
  }
}

In [72]:
ask("which bond should i buy for $100?")

<OpenAIObject text_completion id=cmpl-7GoymhjbIw8SLBbexB9hk99AbJqFR at 0x136d04810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " For $100, you should buy UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/18/2023, as it has the lowest price per $100."
    }
  ],
  "created": 1684243568,
  "id": "cmpl-7GoymhjbIw8SLBbexB9hk99AbJqFR",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 39,
    "prompt_tokens": 926,
    "total_tokens": 965
  }
}

In [15]:
ask("Give me top 5 bonds based on price per $100.")

'="912810SN9"N/AUNITED STATES TREAS SER BONDS           1.25000% 05/15/2050 BDS1.25005/15/2050AAA--56.0663.8943.8873.88720,000(250)10,000(200)CP D \n* ="912810RS9"N/AUNITED STATES TREAS SER BONDS OF MAY 2042.50000% 05/15/2046 BDS2.50005/15/2046AAA--77.7203.9893.9813.9815,000(200)5,000(200)CP D \n* ="912810SR0"N/AUNITED STATES TREAS SER BONDS           1.12500% 05/15/2040 BDS1.12505/15/2040AAANR65.7043.9053.8963.8965,000(200)5,000(200)CP D \n* ="912810SF6"N/AUNITED STATES TREAS SER BONDS           3.00000% 02/15/2049 BDS3.00002/15/2049AAA--85.2693.9133.9053.90520,000(250)5,000(200)CP D \n* ="912810TF5"N/AUNITED STATES TREAS SER BONDS OF FEBRUAR2.37500% 02/15/2042 BDS2.37502/15/2042AAA--79.1253.9633.9573.95720,000(250)5,000(200)CP D \n* ="912810SH2"N/AUNITED STATES TREAS SER BONDS           2.87500% 05/15/2049 BDS2.87505/15/2049AAA--83.3013.9033.9003.9005,000(200)5,000(200)CP D \n* ="912810RQ3"N/AUNITED STATES TREAS'

In [74]:
ask("What is the date of maturity and price per $100 for the bond with CUSIPid = 912810TN8?")

<OpenAIObject text_completion id=cmpl-7GpT1rxf0c3grMygVfJ4hppqluvt2 at 0x136d00900> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The bond with CUSIPid = 912810TN8 has a date of maturity on 02/15/2053 and a price per $100 of 96.124."
    }
  ],
  "created": 1684245443,
  "id": "cmpl-7GpT1rxf0c3grMygVfJ4hppqluvt2",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 952,
    "total_tokens": 988
  }
}

In [75]:
ask("What bond has the highest price per $100?")

<OpenAIObject text_completion id=cmpl-7GpTkA7tRiahyRa323RNaPrIRlQl1 at 0x172a0c130> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The bond with the highest price per $100 is 912803BC6 - UNITED STATES TREAS BD STRIPP ZERO CPN 0.00000% 08/15/2023."
    }
  ],
  "created": 1684245488,
  "id": "cmpl-7GpTkA7tRiahyRa323RNaPrIRlQl1",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 38,
    "prompt_tokens": 927,
    "total_tokens": 965
  }
}

In [76]:
ask("Which bond has the closest date of maturing from 17 March 2023?")

<OpenAIObject text_completion id=cmpl-7GpUODm0i5BxYScOQ6boY43SeqZRd at 0x171841bd0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The bonds that have the closest date of maturing from 17 March 2023 are:\n\n1. UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/15/2023\n2. UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03/20/2023"
    }
  ],
  "created": 1684245528,
  "id": "cmpl-7GpUODm0i5BxYScOQ6boY43SeqZRd",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 66,
    "prompt_tokens": 976,
    "total_tokens": 1042
  }
}

In [77]:
ask("Which bond has the least security term?")

<OpenAIObject text_completion id=cmpl-7GpV8lsTVRwAvzJSbsaVupmpPOeE2 at 0x13644da40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " The bond with the least security term is UNITED STATES TREAS BILLS ZERO CPN 0.00000% 05/18/2023."
    }
  ],
  "created": 1684245574,
  "id": "cmpl-7GpV8lsTVRwAvzJSbsaVupmpPOeE2",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 934,
    "total_tokens": 964
  }
}